In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\WINDOW 10\Desktop\Labs\Datasets\clean_movies_dataset.csv")


In [7]:
df.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,status,...,genre_names,production_companies_ids,production_companies_names,country,release_month,return,collection_id,collection_name,collection_poster,collection_backdrop
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,English,Released,...,"['Animation', 'Comedy', 'Family']",[3],['Pixar Animation Studios'],United States of America,10,12.451801,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,English,Released,...,"['Adventure', 'Fantasy', 'Family']","[559, 2550, 10201]","['TriStar Pictures', 'Teitler Film', 'Intersco...",United States of America,12,4.043035,NaN,NaN,NaN,NaN


In [34]:
meses_esp = {
    'January': 'enero',
    'February': 'febrero',
    'March': 'marzo',
    'April': 'abril',
    'May': 'mayo',
    'June': 'junio',
    'July': 'julio',
    'August': 'agosto',
    'September': 'septiembre',
    'October': 'octubre',
    'November': 'noviembre',
    'December': 'diciembre'
}

dias_semana = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

In [14]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_month'] = df['release_date'].dt.month_name()
df['release_day'] = df['release_date'].dt.day_name()

In [82]:
def peliculas_mes(mes):
    fechas=pd.to_datetime(df['release_date'],format='%Y-%m-%d')
    nmes=fechas[fechas.dt.month_name(locale='es_CO')==mes.capitalize()]
    respuesta=nmes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}

In [83]:
peliculas_mes('Marzo')

{'mes': 'Marzo', 'cantidad': 3553}

In [93]:
def peliculas_dia(dia):
    fechas=pd.to_datetime(df['release_date'],format='%Y-%m-%d')
    ndia=fechas[fechas.dt.day_name(locale='es_CO')==dia.capitalize()]
    respuesta=ndia.shape[0]
    return {'dia':dia, 'cantidad':respuesta}

In [94]:
peliculas_dia('Lunes')

{'dia': 'Lunes', 'cantidad': 3503}

In [46]:
def franquicia(franquicia):
    # Obtiene la cantidad de películas, la ganancia total y el promedio de ganancias para la franquicia especificada
    df_franquicia = df[df['collection_name'] == franquicia]
    cantidad = len(df_franquicia)
    ganancia_total = df_franquicia['revenue'].sum()
    ganancia_promedio = df_franquicia['revenue'].mean()
    return {'franquicia':franquicia, 'cantidad':cantidad, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}

In [47]:
franquicia("Toy Story Collection")


{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1937890605.0,
 'ganancia_promedio': 645963535.0}

In [63]:
def peliculas_pais(pais):
    df_pais = df[df['country'] == pais]
    cantidad = len(df_pais)
    return {'pais':pais, 'cantidad':cantidad}

In [71]:
peliculas_pais("France") 

{'pais': 'France', 'cantidad': 2704}

In [2]:
def productoras(productora):
    #productio_companies_name esta en el dataset como una lista con todas las productoras de la pelicula, por lo que se debe buscar en cada lista si esta la productora
    df_productora = df[df['production_companies_names'].str.contains(productora)]
    cantidad = len(df_productora)
    ganancia_total = df_productora['revenue'].sum()
    return {'productora':productora, 'ganancia_total':ganancia_total, 'cantidad':cantidad}

In [3]:
productoras('TriStar Pictures')

{'productora': 'TriStar Pictures',
 'ganancia_total': 8022366764.0,
 'cantidad': 196}

In [5]:
df['release_year'] = df['release_year'].astype(str)

In [6]:
def retorno(pelicula):
    df_pelicula = df[df['title'] == pelicula]
    inversion = df_pelicula['budget'].values[0]
    ganancia = df_pelicula['revenue'].values[0]-df_pelicula['budget'].values[0]
    retorno = df_pelicula['return'].values[0]
    anio = df_pelicula['release_year'].values[0]
    return {'pelicula':pelicula, 'inversion':inversion, 'ganacia':ganancia,'retorno':retorno, 'año':anio}

In [7]:
retorno("Toy Story")

{'pelicula': 'Toy Story',
 'inversion': 30000000.0,
 'ganacia': 343554033.0,
 'retorno': 12.4518011,
 'año': '1995'}

In [17]:
import ast 
from sklearn.neighbors import NearestNeighbors

In [40]:
df = pd.read_csv(r'C:\Users\WINDOW 10\Desktop\Labs\Datasets\clean_movies_dataset.csv')

In [51]:
import pandas as pd
import ast
from sklearn.neighbors import NearestNeighbors

def ml_movie(selected_title):
    df = pd.read_csv(r'C:\Users\WINDOW 10\Desktop\Labs\Datasets\clean_movies_dataset.csv')
    df['tagline'] = df['tagline'].fillna('')
    df = df.dropna(subset=['overview', 'tagline', 'genre_names', 'title', 'id'])
    
    generos_df = df['genre_names'].str.join(sep='|').str.get_dummies()
    
    selected_genres = df.loc[df['title'] == selected_title]['genre_names'].values
    if len(selected_genres) == 0:
        return "No se encontró la película " + selected_title
    selected_genres = ast.literal_eval(selected_genres[0])
    df['genre_similarity'] = df['genre_names'].apply(lambda x: len(set(selected_genres) & set(ast.literal_eval(x))) / len(set(selected_genres) | set(ast.literal_eval(x))))
    
    df['same_series'] = df['title'].apply(lambda x: 1 if selected_title.lower() in x.lower() else 0)
    
    features_df = pd.concat([generos_df, df['vote_average'], df['genre_similarity'], df['same_series']], axis=1)

    k = 6
    knn = NearestNeighbors(n_neighbors=k+1, algorithm='auto')
    knn.fit(features_df)
    indices = knn.kneighbors(features_df.loc[df['title'] == selected_title])[1].flatten()
    recommended_movies = list(df.iloc[indices]['title'])

    selected_score = df.loc[df['title'] == selected_title]['vote_average'].values[0]
    recommended_movies = sorted(recommended_movies, key=lambda x: (df.loc[df['title'] == x]['same_series'].values[0], df.loc[df['title'] == x]['vote_average'].values[0], df.loc[df['title'] == x]['genre_similarity'].values[0]), reverse=True)
    recommended_movies = [movie for movie in recommended_movies if movie != selected_title]

    if len(recommended_movies) == 0:
        return "No se encontraron películas similares a " + selected_title
    else:
        respuesta = f"Película seleccionada: {selected_title} ({selected_score}):\n\nPelículas Recomendadas:\n"
        for i, pelicula in enumerate(recommended_movies[:5]):
            score = df.loc[df['title'] == pelicula]['vote_average'].values[0]
            genres = df.loc[df['title'] == pelicula]['genre_names'].values[0]
            gen_str = ', '.join(ast.literal_eval(genres))

            respuesta += f"-{pelicula}  | Géneros: {gen_str} | Puntaje: {score} |\n"
            if i == 4:
                break
        return respuesta


In [52]:
ml_movie("Batman")

'Película seleccionada: Batman (7.0):\n\nPelículas Recomendadas:\n-Batman Returns  | Géneros: Action, Fantasy | Puntaje: 6.6 |\n-A Chinese Ghost Story  | Géneros: Fantasy, Action | Puntaje: 7.2 |\n-The Legend of the Swordsman  | Géneros: Action, Fantasy | Puntaje: 6.9 |\n-Spider-Man  | Géneros: Fantasy, Action | Puntaje: 6.8 |\n-Journey to the Center of the Earth  | Géneros: Adventure, Fantasy, Family | Puntaje: 6.7 |\n'

In [ ]:
import pandas as pd
import ast
from sklearn.neighbors import NearestNeighbors

def Movies_ML(selected_title):
    df = pd.read_csv('clean_movies_dataset.csv')
    df['tagline'] = df['tagline'].fillna('')
    df = df.dropna(subset=['overview', 'tagline', 'genre_names', 'title', 'id'])
    
    generos_df = df['genre_names'].str.join(sep='|').str.get_dummies()
    
    selected_genres = df.loc[df['title'] == selected_title]['genre_names'].values
    if len(selected_genres) == 0:
        return "No se encontró la película " + selected_title
    selected_genres = ast.literal_eval(selected_genres[0])
    df['genre_similarity'] = df['genre_names'].apply(lambda x: len(set(selected_genres) & set(ast.literal_eval(x))) / len(set(selected_genres) | set(ast.literal_eval(x))))
    
    df['same_series'] = df['title'].apply(lambda x: 1 if selected_title.lower() in x.lower() else 0)
    
    features_df = pd.concat([generos_df, df['vote_average'], df['genre_similarity'], df['same_series']], axis=1)

    k = 6
    knn = NearestNeighbors(n_neighbors=k+1, algorithm='auto')
    knn.fit(features_df)
    indices = knn.kneighbors(features_df.loc[df['title'] == selected_title])[1].flatten()
    recommended_movies = list(df.iloc[indices]['title'])

    selected_score = df.loc[df['title'] == selected_title]['vote_average'].values[0]
    recommended_movies = sorted(recommended_movies, key=lambda x: (df.loc[df['title'] == x]['same_series'].values[0], df.loc[df['title'] == x]['vote_average'].values[0], df.loc[df['title'] == x]['genre_similarity'].values[0]), reverse=True)
    recommended_movies = [movie for movie in recommended_movies if movie != selected_title]

    if len(recommended_movies) == 0:
        return "No se encontraron películas similares a " + selected_title
    else:
        output_str = f"Película seleccionada: {selected_title} ({selected_score}):\n\nPelículas Recomendadas:\n"
        for i, pelicula in enumerate(recommended_movies[:5]):
            score = df.loc[df['title'] == pelicula]['vote_average'].values[0]
            genres = df.loc[df['title'] == pelicula]['genre_names'].values[0]
            gen_str = ', '.join(ast.literal_eval(genres))

            output_str += f"-{pelicula}  | Géneros: {gen_str} | Puntaje: {score} |\n"
            if i == 4:
                break
        return output_str


In [63]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def ml_movie(selected_title):
    df = pd.read_csv(r'C:\Users\WINDOW 10\Desktop\Labs\Datasets\clean_movies_dataset.csv')
    df = df.dropna(subset=['overview', 'tagline', 'genre_names', 'title', 'id'])

    generos_df = df['genre_names'].str.join(sep='|').str.get_dummies()

    selected_title = "Batman"
    selected_genres = df.loc[df['title'] == selected_title, 'genre_names'].values[0]
    df['genre_similarity'] = df['genre_names'].apply(lambda x: len(set(selected_genres) & set(x)) / len(set(selected_genres) | set(x)))
    df['same_series'] = df['title'].apply(lambda x: 1 if 'Batman' in x else 0)

    features_df = pd.concat([generos_df, df['vote_average'], df['genre_similarity'], df['same_series']], axis=1)

    k = 6
    knn = NearestNeighbors(n_neighbors=k+1, algorithm='auto')
    knn.fit(features_df)
    indices = knn.kneighbors(features_df.loc[df['title'] == selected_title])[1].flatten()
    recommended_movies = list(df.iloc[indices]['title'])

    selected_score = df.loc[df['title'] == selected_title, 'vote_average'].values[0]
    recommended_movies = sorted(recommended_movies, key=lambda x: (df.loc[df['title'] == x, 'same_series'].values[0], df.loc[df['title'] == x, 'vote_average'].values[0], df.loc[df['title'] == x, 'genre_similarity'].values[0]), reverse=True)
    recommended_movies = [movie for movie in recommended_movies if movie != selected_title]
    
    print(f"Pelicula seleccionada: {selected_title} ({selected_score}):\n")
    print("Peliculas Recomendadas:\n")
    for i, pelicula in enumerate(recommended_movies[:5]):
        score = df.loc[df['title'] == pelicula, 'vote_average'].values[0]
        genres = df.loc[df['title'] == pelicula, 'genre_names'].values[0]
        gen_str = ', '.join(genres)
        print(f"-{pelicula}  | Generos: {gen_str} | Puntaje: {score} |")
        if i == 4:
            break
    print("\n")
    print(recommended_movies[0:5])

In [64]:
ml_movie("Batman")

Pelicula seleccionada: Batman (7.0):

Peliculas Recomendadas:

-Batman Returns  | Generos: [, ', A, c, t, i, o, n, ', ,,  , ', F, a, n, t, a, s, y, ', ] | Puntaje: 6.6 |
-Justice League: Gods and Monsters  | Generos: [, ', A, c, t, i, o, n, ', ,,  , ', A, n, i, m, a, t, i, o, n, ', ,,  , ', F, a, n, t, a, s, y, ', ] | Puntaje: 6.9 |
-Spider-Man  | Generos: [, ', F, a, n, t, a, s, y, ', ,,  , ', A, c, t, i, o, n, ', ] | Puntaje: 6.8 |
-Journey to the Center of the Earth  | Generos: [, ', A, d, v, e, n, t, u, r, e, ', ,,  , ', F, a, n, t, a, s, y, ', ,,  , ', F, a, m, i, l, y, ', ] | Puntaje: 6.7 |
-The Challenge  | Generos: [, ', A, c, t, i, o, n, ', ,,  , ', S, c, i, e, n, c, e,  , F, i, c, t, i, o, n, ', ] | Puntaje: 6.6 |


['Batman Returns', 'Justice League: Gods and Monsters', 'Spider-Man', 'Journey to the Center of the Earth', 'The Challenge']
